# Overall Result and Questions

A handoff on a run play from the quarterback to the ball carrier should take place behind the line of scrimmage. So if we tracked the ball on the x coordinate from the snap to the handoff the net yardage should be negative. To track where the ball is at any one time I'll track where the ball is on the x coordinate for the ball carrier in each frame of each play. 

# The Analysis

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None


To do this comparison I'll need the playResult from the plays dataset. The best way I can see to track the ball across frames is to track where the ball carrier is at on the x coordinate in each frame, and I'll need the plays dataset for the ballCarrierId information as well.

To keep this simple, I'll just do this comparison for the first week. I'm assuming whatever I find will be reflected in the other 8 weeks. 

In [2]:
plays = pd.read_csv('./data/plays.csv')
week1 = pd.read_csv('./data/tracking_week_1.csv')

In [3]:
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,ATL,41,7:52,21,7,C,6.0,NaN,9,9,N,69,SHOTGUN,7.0,0.747284,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,PIT,34,7:38,14,20,NaN,NaN,NaN,3,3,N,76,SHOTGUN,7.0,0.416454,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,LV,30,8:57,10,3,C,11.0,NaN,15,15,N,40,I_FORM,6.0,0.267933,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,DEN,37,13:12,19,16,NaN,NaN,NaN,7,7,N,47,SINGLEBACK,6.0,0.592704,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,TEN,35,8:33,7,7,NaN,NaN,NaN,3,3,N,75,I_FORM,7.0,0.470508,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,WAS,39,13:31,0,0,NaN,NaN,NaN,1,1,N,49,SINGLEBACK,6.0,0.577800,0.638600,0.361400,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,SEA,30,6:08,17,16,C,0.0,NaN,2,2,N,40,SINGLEBACK,7.0,0.298983,0.615241,0.384759,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,CIN,41,9:35,26,21,NaN,NaN,NaN,6,6,N,69,SHOTGUN,6.0,0.639439,0.667054,0.332946,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,TEN,34,2:02,17,24,NaN,NaN,NaN,-2,-2,N,44,SHOTGUN,6.0,0.518695,0.410611,0.589389,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN


Since playId is "...not unique across games" I'll combine that column and gameId to create a new variable called gamePlayId to have a unique identifier for each play. 

In [4]:
plays['gamePlayId'] = plays[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
plays['gamePlayId'] = plays['gamePlayId'].astype('int64')

I'll do the same thing on the week1 dataset. 

In [5]:
week1['gamePlayId'] = week1[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
week1['gamePlayId'] = week1['gamePlayId'].astype('int64')

In [6]:
week1['gamePlayFrameId'] = week1[['gamePlayId', 'frameId']].astype(str).apply(lambda x: ''.join(x), axis=1)
week1['gamePlayFrameId'] = week1['gamePlayFrameId'].astype('int64')

To only include the columns I need from the plays dataset to merge into the week1 dataset. 

In [7]:
plays_merge = plays[['gamePlayId', 'ballCarrierId']]

In [8]:
plays_merge

,gamePlayId,ballCarrierId,playResult
0,20221009083537,48723,9
1,20220911033126,52457,3
2,20220911111148,42547,15
3,20221002122007,46461,7
4,20220919001372,47857,3
...,...,...,...
12481,2022100204123,43293,1
12482,20220912003467,46189,2
12483,20221016053371,44860,6
12484,20221002072777,52449,-2


In [9]:
week1 = pd.merge(week1, plays_merge, on=['gamePlayId'], how='left')

In [10]:
week1

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,gamePlayId,gamePlayFrameId,ballCarrierId,playResult
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,2022090800561,42489,6
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,2022090800562,42489,6
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,2022090800563,42489,6
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,2022090800564,42489,6
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,2022090800565,42489,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,2022-09-12 23:05:57.799999,NaN,football,left,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,2022091200382649,53464,9
1407435,2022091200,3826,NaN,football,50,2022-09-12 23:05:57.900000,NaN,football,left,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,2022091200382650,53464,9
1407436,2022091200,3826,NaN,football,51,2022-09-12 23:05:58.000000,NaN,football,left,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,2022091200382651,53464,9
1407437,2022091200,3826,NaN,football,52,2022-09-12 23:05:58.099999,NaN,football,left,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,2022091200382652,53464,9


In order to track where the ball is on any frame, I'll want to find each frame where the nflId is equal to the ballCarrierId. However, there are 61,193 rows where the nflId is 'NaN'. These correspond to where the displayName is 'football'. So for this exercise, I'll just fill the "NaN' nflId's with zeros. 

In [11]:
week1['nflId'].isna().sum()

61193

In [12]:
week1['displayName'].value_counts()

football             61193
Josh Allen            3280
David Long            3261
Jalen Pitre           2854
Jonathan Owens        2854
                     ...  
Keisean Nixon           20
P.J. Locke              16
Eric Johnson            15
Jonathan Williams       14
J.C. Hassenauer         13
Name: displayName, Length: 1166, dtype: int64

In [13]:
week1['nflId'] = week1['nflId'].fillna(0)
week1['nflId'] = week1['nflId'].astype(int)

Here we see that there are almost 27,000 plays where a tackle is made, but only 16,000 plays where the ball is snapped. 

In [14]:
week1['event'].value_counts()

first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

I will therefore eliminate all the ball_snap events and just set the snap to occur on the first frame. While the snap count doesn't always occur on the first frame, I am assuming any activities occuring on the frames before the snap are simply pre-snap movements where the ball stays at the same location. 

In [15]:
week1 = week1[week1.event !='ball_snap']

In [16]:
week1['event'] = np.where(week1['frameId']==1, 'snap', week1['event'])

The new variable xBallLocation will be the x coordinate of that player where that player is the ball carrier. That should be the yard line the ball is on for each frame of a particular play. 

In [17]:
week1['xBallLocation'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['x'], 0)

In [18]:
week1['xBallLocation'] = week1.groupby('gamePlayFrameId')['xBallLocation'].transform(max)

Where the ball is for the snap and for the handoff will be the ball location for the frame the where the snap or the handoff occurs. I'll then spread those values to all the plays on that frame. 

In [27]:
week1['xSnap'] = np.where(week1['event'] == 'snap', week1['xBallLocation'], 0)
week1['xHandoff'] = np.where(week1['event'] == 'handoff', week1['xBallLocation'], 0)

In [33]:
week1['xSnap'] = week1.groupby('gamePlayId')['xSnap'].transform(max)
week1['xHandoff'] = week1.groupby('gamePlayId')['xHandoff'].transform(max)

In [34]:
week1

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,gamePlayId,gamePlayFrameId,ballCarrierId,playResult,xBallLocation,xSnap,xHandoff
0,2022090800,56,35472,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,2022090800561,42489,6,80.60,80.60,0.0
1,2022090800,56,35472,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,2022090800562,42489,6,80.48,80.60,0.0
2,2022090800,56,35472,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,2022090800563,42489,6,80.34,80.60,0.0
3,2022090800,56,35472,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,2022090800564,42489,6,80.19,80.60,0.0
4,2022090800,56,35472,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,2022090800565,42489,6,80.01,80.60,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,2022-09-12 23:05:57.799999,NaN,football,left,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,2022091200382649,53464,9,56.06,70.71,0.0
1407435,2022091200,3826,0,football,50,2022-09-12 23:05:57.900000,NaN,football,left,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,2022091200382650,53464,9,55.96,70.71,0.0
1407436,2022091200,3826,0,football,51,2022-09-12 23:05:58.000000,NaN,football,left,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,2022091200382651,53464,9,55.87,70.71,0.0
1407437,2022091200,3826,0,football,52,2022-09-12 23:05:58.099999,NaN,football,left,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,2022091200382652,53464,9,55.79,70.71,0.0


In [47]:
week1['gamePlayId'].nunique()

1473

In [48]:
week1['xSnap'].value_counts()

69.14    3864
29.29    3565
93.18    3496
41.86    3082
84.33    3059
         ... 
67.23     253
50.88     253
63.70     253
79.82     253
46.10     253
Name: xSnap, Length: 1373, dtype: int64

In [49]:
week1['xHandoff'].value_counts()

0.00      567318
45.54       3381
54.64       3128
58.72       3036
84.05       3013
           ...  
69.88        713
105.65       690
45.28        690
21.83        690
13.44        598
Name: xHandoff, Length: 650, dtype: int64

Since handoffs don't occur on every play, I'll create a dataframe that only includes those plays that have handoffs. Other plays will be pass plays or direct plays with no transfer of the ball from the quarterback or pistol back. 

In [40]:
week1_handoffs = week1[week1.xHandoff !=0]

In [42]:
week1_handoffs

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,gamePlayId,gamePlayFrameId,ballCarrierId,playResult,xBallLocation,xSnap,xHandoff
1196,2022090800,101,35472,Rodger Saffold,1,2022-09-08 20:25:08.200000,76.0,BUF,left,73.130000,27.640000,0.01,0.01,0.00,267.95,182.22,snap,2022090800101,20220908001011,47857,7,79.61,79.61,79.16
1197,2022090800,101,35472,Rodger Saffold,2,2022-09-08 20:25:08.299999,76.0,BUF,left,73.130000,27.640000,0.01,0.01,0.00,267.95,182.56,NaN,2022090800101,20220908001012,47857,7,79.60,79.61,79.16
1198,2022090800,101,35472,Rodger Saffold,3,2022-09-08 20:25:08.400000,76.0,BUF,left,73.130000,27.630000,0.01,0.01,0.00,267.95,182.15,NaN,2022090800101,20220908001013,47857,7,79.60,79.61,79.16
1199,2022090800,101,35472,Rodger Saffold,4,2022-09-08 20:25:08.500000,76.0,BUF,left,73.130000,27.640000,0.00,0.01,0.01,274.37,179.54,NaN,2022090800101,20220908001014,47857,7,79.60,79.61,79.16
1200,2022090800,101,35472,Rodger Saffold,5,2022-09-08 20:25:08.599999,76.0,BUF,left,73.130000,27.650000,0.00,0.25,0.01,274.95,1.03,NaN,2022090800101,20220908001015,47857,7,79.60,79.61,79.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404214,2022091200,3685,0,football,54,2022-09-12 22:59:01.099999,NaN,football,left,70.889999,29.160000,1.92,1.65,0.18,NaN,NaN,tackle,20220912003685,2022091200368554,42358,2,70.88,76.82,76.25
1404215,2022091200,3685,0,football,55,2022-09-12 22:59:01.200000,NaN,football,left,71.040001,29.299999,2.06,1.44,0.20,NaN,NaN,NaN,20220912003685,2022091200368555,42358,2,71.02,76.82,76.25
1404216,2022091200,3685,0,football,56,2022-09-12 22:59:01.299999,NaN,football,left,71.279999,29.469999,2.50,1.73,0.30,NaN,NaN,NaN,20220912003685,2022091200368556,42358,2,71.18,76.82,76.25
1404217,2022091200,3685,0,football,57,2022-09-12 22:59:01.400000,NaN,football,left,71.480003,29.629999,2.61,1.33,0.25,NaN,NaN,NaN,20220912003685,2022091200368557,42358,2,71.35,76.82,76.25


To calculate the distance the ball moved from the snap to the handoff. 

In [43]:
week1_handoffs['xSnapToHandoff'] = week1_handoffs['xHandoff'] - week1_handoffs['xSnap']

In [44]:
week1_handoffs

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,gamePlayId,gamePlayFrameId,ballCarrierId,playResult,xBallLocation,xSnap,xHandoff,xSnapToHandoff
1196,2022090800,101,35472,Rodger Saffold,1,2022-09-08 20:25:08.200000,76.0,BUF,left,73.130000,27.640000,0.01,0.01,0.00,267.95,182.22,snap,2022090800101,20220908001011,47857,7,79.61,79.61,79.16,-0.45
1197,2022090800,101,35472,Rodger Saffold,2,2022-09-08 20:25:08.299999,76.0,BUF,left,73.130000,27.640000,0.01,0.01,0.00,267.95,182.56,NaN,2022090800101,20220908001012,47857,7,79.60,79.61,79.16,-0.45
1198,2022090800,101,35472,Rodger Saffold,3,2022-09-08 20:25:08.400000,76.0,BUF,left,73.130000,27.630000,0.01,0.01,0.00,267.95,182.15,NaN,2022090800101,20220908001013,47857,7,79.60,79.61,79.16,-0.45
1199,2022090800,101,35472,Rodger Saffold,4,2022-09-08 20:25:08.500000,76.0,BUF,left,73.130000,27.640000,0.00,0.01,0.01,274.37,179.54,NaN,2022090800101,20220908001014,47857,7,79.60,79.61,79.16,-0.45
1200,2022090800,101,35472,Rodger Saffold,5,2022-09-08 20:25:08.599999,76.0,BUF,left,73.130000,27.650000,0.00,0.25,0.01,274.95,1.03,NaN,2022090800101,20220908001015,47857,7,79.60,79.61,79.16,-0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404214,2022091200,3685,0,football,54,2022-09-12 22:59:01.099999,NaN,football,left,70.889999,29.160000,1.92,1.65,0.18,NaN,NaN,tackle,20220912003685,2022091200368554,42358,2,70.88,76.82,76.25,-0.57
1404215,2022091200,3685,0,football,55,2022-09-12 22:59:01.200000,NaN,football,left,71.040001,29.299999,2.06,1.44,0.20,NaN,NaN,NaN,20220912003685,2022091200368555,42358,2,71.02,76.82,76.25,-0.57
1404216,2022091200,3685,0,football,56,2022-09-12 22:59:01.299999,NaN,football,left,71.279999,29.469999,2.50,1.73,0.30,NaN,NaN,NaN,20220912003685,2022091200368556,42358,2,71.18,76.82,76.25,-0.57
1404217,2022091200,3685,0,football,57,2022-09-12 22:59:01.400000,NaN,football,left,71.480003,29.629999,2.61,1.33,0.25,NaN,NaN,NaN,20220912003685,2022091200368557,42358,2,71.35,76.82,76.25,-0.57


# The Results

The first 15 plays show all negative numbers, with some exaggerated results. I'm not sure why some of these are so large. 

In [45]:
play_yards = week1_handoffs.groupby('gamePlayId').agg(
    PlayResult = ('xSnapToHandoff', max),    
    
).reset_index()

play_yards_sorted = play_yards.sort_values(by='gamePlayId')
play_yards_sorted.head(15)

,gamePlayId,PlayResult
0,202209110078,-2.78
1,202209110286,-0.17
2,202209110486,-2.39
3,202209110556,-0.12
4,202209110683,-16.96
5,202209110768,-2.59
6,202209110862,1.85
7,202209110991,-2.15
8,202209111055,-0.95
9,202209111286,1.64


Five of the 16 games have positive average yardage on the handoffs. I'm not sure why that is occuring as well. 

In [46]:
play_yards = week1_handoffs.groupby('gameId').agg(
    PlayResult = ('xSnapToHandoff', 'mean'), 
    
).reset_index()

play_yards_sorted = play_yards.sort_values(by='gameId')
play_yards_sorted

,gameId,PlayResult
0,2022090800,-0.455809
1,2022091100,-0.588372
2,2022091101,-1.082036
3,2022091102,-0.222968
4,2022091103,-0.052760
5,2022091104,-0.168634
6,2022091105,-0.137447
7,2022091106,-0.507281
8,2022091107,0.217378
9,2022091108,0.229699
